In [63]:
import numpy as np
import pandas as pd
%matplotlib inline

In [64]:
data = pd.read_csv("from_06_01_5_minute_stat.csv")
data.set_index("FROM_TIME", inplace=True)
data.head(10)

,END_TIME,AVG_RUNNING,TRIPS_COUNT,VEHICLE_COUNT
FROM_TIME,,,,
2018-06-01 00:05:00,2018-06-01 00:10:00,16.22,18,186
2018-06-01 00:10:00,2018-06-01 00:15:00,16.65,21,180
2018-06-01 00:15:00,2018-06-01 00:20:00,18.88,23,197
2018-06-01 00:20:00,2018-06-01 00:25:00,18.97,26,203
2018-06-01 00:25:00,2018-06-01 00:30:00,20.68,29,212
2018-06-01 00:30:00,2018-06-01 00:35:00,20.09,40,222
2018-06-01 00:35:00,2018-06-01 00:40:00,19.73,33,228
2018-06-01 00:40:00,2018-06-01 00:45:00,19.44,30,216
2018-06-01 00:45:00,2018-06-01 00:50:00,20.19,40,226


# 根据输入的次数获取预估发车量
数据为当前到之前n-1个数据之和

In [65]:
def getEstTripsCount(df,n=5):
    trips_count = df["TRIPS_COUNT"]
    estTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if i < n-1:
            estTripsCount.append(0)
        else:
            estTripsCount.append(sum(trips_count[i+1-n:i+1]))
    return estTripsCount


In [66]:
estTripsCount_20 = getEstTripsCount(data, n=4)
estTripsCount_25 = getEstTripsCount(data, n=5)
estTripsCount_30 = getEstTripsCount(data, n=6)
estTripsCount_35 = getEstTripsCount(data, n=7)
estTripsCount_40 = getEstTripsCount(data, n=8)

# 计算真实发车辆
计算方法为获取当前时间点之后的n个点trips_count之和

In [67]:
def getRealTripsCount(df,n=5):
    trips_count = df["TRIPS_COUNT"]
    realTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if l-i-1 < n:
            realTripsCount.append(0)
        else:
            realTripsCount.append(sum(trips_count[i+1:i+1+n]))
    return realTripsCount

In [68]:
realTripsCount_20 = getRealTripsCount(data, n=4)
realTripsCount_25 = getRealTripsCount(data, n=5)
realTripsCount_30 = getRealTripsCount(data, n=6)
realTripsCount_35 = getRealTripsCount(data, n=7)
realTripsCount_40 = getRealTripsCount(data, n=8)

# 计算空车差量 5%，10%，15%

In [69]:
data["VEHICLE_COUNT_105"] = data.VEHICLE_COUNT*1.05
data["VEHICLE_COUNT_110"] = data.VEHICLE_COUNT*1.10
data["VEHICLE_COUNT_115"] = data.VEHICLE_COUNT*1.15

# 合并数据集
将计算的预估发车辆、真实发车量数据合并到Data数据集中，命名为data_final

In [70]:
trips_count_dict = {"estTripsCount_20":estTripsCount_20,
                    "estTripsCount_25":estTripsCount_25,
                    "estTripsCount_30":estTripsCount_30,
                    "estTripsCount_35":estTripsCount_35,
                    "estTripsCount_40":estTripsCount_40,
                    "realTripsCount_20":realTripsCount_20,
                    "realTripsCount_25":realTripsCount_25,
                    "realTripsCount_30":realTripsCount_30,
                    "realTripsCount_35":realTripsCount_35,
                    "realTripsCount_40":realTripsCount_40                    
                   }

In [71]:
data_trips_count = pd.DataFrame(data=trips_count_dict, index=data.index)

In [72]:
data_final = data.merge(data_trips_count, left_index=True, right_index=True)

In [73]:
data_final.head(1)

,END_TIME,AVG_RUNNING,TRIPS_COUNT,VEHICLE_COUNT,VEHICLE_COUNT_105,VEHICLE_COUNT_110,VEHICLE_COUNT_115,estTripsCount_20,estTripsCount_25,estTripsCount_30,estTripsCount_35,estTripsCount_40,realTripsCount_20,realTripsCount_25,realTripsCount_30,realTripsCount_35,realTripsCount_40
FROM_TIME,,,,,,,,,,,,,,,,,
2018-06-01 00:05:00,2018-06-01 00:10:00,16.22,18,186,195.3,204.6,213.9,0,0,0,0,0,99,139,172,202,242


# 获取预估发车辆（预估和真实）大于空车数差量的数据
以目标蓄车时间20min、差量5%为例的计算结果

### 1.获取数据集

In [74]:
data_final_est20_lg_105 = data_final[data_final["estTripsCount_20"]>data_final["VEHICLE_COUNT_105"]]
data_final_real20_lg_105 = data_final[data_final["realTripsCount_20"]>data_final["VEHICLE_COUNT_105"]]
data_final_both20_lg_105 = data_final[(data_final["estTripsCount_20"]>data_final["VEHICLE_COUNT_105"])&(data_final["realTripsCount_20"]>data_final["VEHICLE_COUNT_105"])]


### 2.获取各数据集数据数量

In [75]:
est_20_105 = data_final_est20_lg_105.END_TIME.count()
est_20_105

158

In [89]:
data_final_est20_lg_105["END_TIME"] = pd.to_datetime(data_final_est20_lg_105["END_TIME"])
data_final_est20_lg_105.head()


c:\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,END_TIME,AVG_RUNNING,TRIPS_COUNT,VEHICLE_COUNT,VEHICLE_COUNT_105,VEHICLE_COUNT_110,VEHICLE_COUNT_115,estTripsCount_20,estTripsCount_25,estTripsCount_30,estTripsCount_35,estTripsCount_40,realTripsCount_20,realTripsCount_25,realTripsCount_30,realTripsCount_35,realTripsCount_40
FROM_TIME,,,,,,,,,,,,,,,,,
2018-06-01 17:10:00,2018-06-01 17:15:00,26.63,22,77,80.85,84.7,88.55,81,98,109,121,135,91,117,131,141,152
2018-06-01 17:15:00,2018-06-01 17:20:00,26.45,24,76,79.80,83.6,87.40,85,105,122,133,145,93,107,117,128,144
2018-06-01 17:20:00,2018-06-01 17:25:00,21.74,21,71,74.55,78.1,81.65,91,106,126,143,154,86,96,107,123,135
2018-06-01 17:25:00,2018-06-01 17:30:00,18.76,21,67,70.35,73.7,77.05,88,112,127,147,164,75,86,102,114,125
2018-06-01 17:30:00,2018-06-01 17:35:00,17.30,25,71,74.55,78.1,81.65,91,113,137,152,172,61,77,89,100,111


In [76]:
real_20_105 = data_final_real20_lg_105.END_TIME.count()
real_20_105

99

In [77]:
both_20_105 = data_final_both20_lg_105.END_TIME.count()
both_20_105

68

### 3.计算上报准确度

In [78]:
accuracy_20_105 = both_20_105/est_20_105
accuracy_20_105

0.43037974683544306

### 4.计算误报率

In [79]:
false_positive_20_105 = (real_20_105-both_20_105)/est_20_105
false_positive_20_105

0.1962025316455696